
# Implementation of cnn network

In [15]:
import torch 
import numpy as np
import pandas  as pd
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
torch.set_printoptions(linewidth=120)


In [127]:
#r""" taking a closer look into the network class


class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels= 1, out_channels= 6 , kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels= 6, out_channels= 12 , kernel_size=5)
        
        self.fc1 = nn.Linear(in_features= 12*4*4 , out_features= 120)
        self.fc2 = nn.Linear(in_features= 120, out_features= 60)
        self.out = nn.Linear(in_features= 60, out_features= 10)

    def forward(self,t):
        
        #1 inpput layer 
        
        t = t
        
        #2 hidden conv layer
        
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size = 2 , stride = 2)
        
        #3 hidden conv layer 
        
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size = 2 , stride = 2)
        
        #4 Linear layer 
        
        t = t.reshape(-1, 12*4*4)     #flattening is hapening here
        t = self.fc1(t)
        t = F.relu(t)

        
        #5 Linear layer 
        
        t = self.fc2(t)
        t = F.relu(t)
        
        #6 output layer
        
        t = self.out(t)
        # t = F.softmax(t,dim = 0)    but this line is not required because we will predict the output later and softmax will be used explicitly later
        
        
        


In [17]:
#loading data from url of Mnist

train_set = torchvision.datasets.FashionMNIST(
        root = './data'
        ,train = True
        ,download=True
        ,transform = transforms.Compose([
            transforms.ToTensor()
        ])
    
)

In [59]:
#Building my own network
r"""
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1 , out_channels= 6, kernel_size= 5)
        self.conv2= nn.Conv2d(in_channels=6, out_channels= 12,kernel_size= 5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60 , out_features= 10)
        
    def forward(self,t):
        
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t,kernel_size= 2 , stride = 2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size = 2 , stride= 2)
        
        t = F.relu(self.fc1(t.reshape(-1 , 12*4*4)))
        t = F.relu(self.fc2(t))
        t = self.out(t)                               
        return t

In [112]:
mynet = Network()

# Inspecting the output tensor for one image sample

In [56]:


sample = next(iter(train_set))
image , label = sample
image.shape


torch.Size([1, 28, 28])

In [57]:
# but my neural network is expecting a rank 4 teensor and in this image there is no batch size given

image.unsqueeze(0).shape # and we do it via unsquuze method

torch.Size([1, 1, 28, 28])

In [61]:
pred = mynet(image.unsqueeze(0))


In [63]:
pred.shape

torch.Size([1, 10])

In [75]:
pred

tensor([[ 0.0425,  0.0017, -0.1016,  0.0987, -0.0355,  0.0827, -0.0817, -0.0182, -0.0463, -0.1511]],
       grad_fn=<AddmmBackward>)

In [64]:
label   #what i am expecting

9

In [81]:
new_pred = F.softmax(pred,dim= 1)
new_pred

tensor([[0.1062, 0.1020, 0.0920, 0.1124, 0.0983, 0.1106, 0.0938, 0.1000, 0.0972, 0.0875]], grad_fn=<SoftmaxBackward>)

In [86]:
new_pred.argmax()   #expected class representing 9 but got 3

tensor(3)

In [87]:
new_pred.sum()

tensor(1., grad_fn=<SumBackward0>)

In [88]:
pred.sum()

tensor(-0.2087, grad_fn=<SumBackward0>)

# passing an entire batch of images to my network

In [90]:
data_loader = torch.utils.data.DataLoader(train_set,
                    batch_size= 10                     
                    )


In [92]:
batch = next(iter(data_loader))

In [94]:
images , labels = batch
images.shape

torch.Size([10, 1, 28, 28])

In [95]:
# this time the data is already in ranked 4 tensor so we dont need to squeeze anything 
labels.shape

torch.Size([10])

In [113]:
preds = mynet(images)

In [114]:
preds.shape

torch.Size([10, 10])

In [115]:
preds =  F.softmax(preds, dim = 1)
preds

tensor([[0.1070, 0.0890, 0.1084, 0.0963, 0.0971, 0.1133, 0.0939, 0.0947, 0.1098, 0.0904],
        [0.1069, 0.0892, 0.1082, 0.0960, 0.0979, 0.1137, 0.0939, 0.0948, 0.1094, 0.0900],
        [0.1072, 0.0892, 0.1084, 0.0959, 0.0981, 0.1135, 0.0938, 0.0945, 0.1090, 0.0903],
        [0.1071, 0.0892, 0.1083, 0.0959, 0.0980, 0.1136, 0.0939, 0.0945, 0.1092, 0.0904],
        [0.1070, 0.0892, 0.1084, 0.0959, 0.0977, 0.1134, 0.0939, 0.0947, 0.1095, 0.0902],
        [0.1066, 0.0891, 0.1084, 0.0961, 0.0979, 0.1139, 0.0939, 0.0950, 0.1092, 0.0898],
        [0.1070, 0.0893, 0.1085, 0.0961, 0.0980, 0.1137, 0.0938, 0.0943, 0.1090, 0.0903],
        [0.1070, 0.0890, 0.1083, 0.0962, 0.0972, 0.1135, 0.0941, 0.0952, 0.1095, 0.0900],
        [0.1069, 0.0893, 0.1084, 0.0958, 0.0982, 0.1137, 0.0939, 0.0944, 0.1091, 0.0903],
        [0.1065, 0.0891, 0.1087, 0.0956, 0.0983, 0.1145, 0.0934, 0.0946, 0.1092, 0.0901]], grad_fn=<SoftmaxBackward>)

In [116]:
preds.argmax(1)   #possible output tensor with classes


tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [117]:
labels   # expected output


tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5])

In [119]:
preds.argmax(dim = 1 ).eq(labels)

tensor([False, False, False, False, False, False, False, False,  True,  True])

In [120]:
preds.argmax(dim = 1).eq(labels).sum()

tensor(2)

In [125]:
def get_currect_number(preds,labels):
    
    return preds.argmax(dim = 1).eq(labels).sum().item()

In [126]:
get_currect_number(preds,labels)

2